In [ ]:
from dorna2 import Dorna
import numpy as np

In [ ]:
###############
# Config Cell #
###############

robot_ip_address = "192.168.1.100"  # the robot ip address
gripper_output = [0, 0, 1]  # gripper output configuration: (pin, off state, on state)
tcp = [0, 0, 0, 0, 0, 0]  # tool center point, tool length and position
positions = {
    "A1": [0, 0, 0, 0, 0, 0],  # position for A1
    "A6": [0, 0, 0, 0, 0, 0],  # position for A6 (example distance)
    "D1": [0, 0, 0, 0, 0, 0],  # position for D1 (example distance)
}

middle_position = [0, 0, 0, 0, 0, 0]  # middle position to hold vial at
safe_joint = [0, 0, 0, 0, 0, 0]  # safe while waiting to pick position
safe_position = []  # j-value of safe joint
speed = 0.5  # set the robot speed

In [ ]:
row_distance = np.divide(np.subtract(positions["D1"], positions["A1"]), 3)  # calculates distance between rows
column_distance = np.divide(np.subtract(positions["A6"], positions["A1"]), 5)  # calculates distance between columns

# Generate positions for the entire grid
grid_positions = {}
rows = ['A', 'B', 'C', 'D']
columns = ['1', '2', '3', '4', '5', '6']

for i, row in enumerate(rows):
    for j, column in enumerate(columns):
        # calculate the position of each cell
        position = np.add(positions["A1"], i * row_distance + j * column_distance)
        grid_positions[f"{row}{column}"] = position.tolist()

In [ ]:
robot = Dorna()
robot.connect(robot_ip_address)

In [ ]:
robot.set_output(gripper_output[0], gripper_output[1])  # set output (pin, state)
robot.set_motor(1)  # turn on the robot motors
robot.jmove(safe_joint)  # Move to a safe initial position

In [ ]:
###############
# Pick and Place Operation #
###############

# prompt user for pick and place positions
pick_location = input("Where do you want to pick from? (e.g., C2): ").upper()
place_location = input("Where do you want to place the vial? (e.g., D6): ").upper()

# ensure valid inputs
if pick_location not in grid_positions or place_location not in grid_positions:
    raise ValueError("Invalid grid position entered. Please use the format 'C2'.")

# get positions from the grid
pick_position = grid_positions[pick_location]
place_position = grid_positions[place_location]

# execute the pick-and-place operation
robot.pick_n_place(
    pick_pose=pick_position,       # pick position from user input
    middle_pose=middle_position,   # middle position to avoid obstacles
    place_pose=place_position,     # place position from user input
    end_pose=safe_position,        # end position (e.g., for imaging)
    output=gripper_output,         # gripper control: [pin_number, pick_state, place_state]
    above=5,                       # above position offset for safe pick/place
    sleep=0.5,                     # delay between operations
    tcp=tcp,                       # tool
    speed=speed                    # movement speed
)

In [ ]:
robot.close() # close the robot connection